In [326]:
## Program to look at regions and region coordinates

In [148]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
HPC_labels,ENT_labels,PHC_labels = getMTLregions(MTL_labels)

exp = 'catFR1'

poss_subs = df[df.experiment == exp]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [149]:
# sub lists I've been using to explore FR1 

# subs = ['R1002P','R1003P','R1006P','R1010J','R1112M','R1163T'] # initial 6 for hippocampus
# subs = subs+['R1001P','R1092J','R1151E','R1320D'] # additional subs with issues that I resolved with various RR and corr thresholds
# subs = ['R1112M','R1018P','R1020J','R1022J','R1023J','R1026D','R1027J'] # 7 with entorhinal
# subs = ['R1010J','R1112M'] # original 2
subs = ['R1112M','R1163T'] # subjects with FR1 and catFR1. SRs of 1000 and 500 tho
# subs = ['R1151E'] # all channels X sessions get removed but 3 after 60/180 hz line removal
# these should all be worked out now. See SWR FR1 problem sessions PPT on Box for details of each
# subs = ['R1299T','R1332M','R1397D','R1349T','R1339D','R1337E','R1275D','R1151E','R1123C','R1120E','R1045E'] # final surrounding_recall problem subs after looking at huge raster!
# subs = ['R1308T','R1338T','R1358T'] # these guys had weird SRs and some loading problems when I went to whole_retrieval analysis...fixed with rounding
# subs = ['R1023J','R1101T','R1148P','R1368T','R1323T','R1334T'] # single session subs with memory allocation errors in ClusterRun
# subs = ['R1006P','R1010J','R1030J','R1032D','R1049J','R1051J','R1052E','R1054J','R1093J','R1098D','R1104D','R1108J','R1111M',
#         'R1115T','R1118N','R1124J','R1207J','R1230J','R1316T','R1329T','R1323T','R1337E','R1367D'] # subs that didn't load on 2020-07-04 encoding run
# subs = ['R1368T','R1461T','R1463E'] # subs with particularly low localization.pair matches in pairs...see Excel in loading info on Box for details
# subs = ['R1433E','R1355T','R1367D','R1368T'] # subs with "ca3" or "dg"
# subs = ['R1065J'] # subject with over 8000 FR1 trials X electrodes
# bad_subs = ['R1433E','R1051J'] # no electrode cats

# sub lists to explore catFR

sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp)] # all sessions for subs
# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp) & (df.session==1)] # single session
# sub_df = df[(df.experiment == exp) & (df.session == 0)] # all FR subs 
# sub_df = sub_df[1:2]
sub_df

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
659,NaN,protocols/r1/subjects/R1112M/experiments/catFR...,protocols/r1/subjects/R1112M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1112M/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1112M/localizations/0/m...,NaN,0,R1112M,R1112M,1.0,protocols/r1/subjects/R1112M/experiments/catFR...
934,NaN,protocols/r1/subjects/R1163T/experiments/catFR...,protocols/r1/subjects/R1163T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1163T/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1163T/localizations/0/m...,NaN,0,R1163T,R1163T,NaN,protocols/r1/subjects/R1163T/experiments/catFR...


In [150]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
exp_df.shape
exp_df

(481, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
24,NaN,protocols/r1/subjects/R1004D/experiments/catFR...,protocols/r1/subjects/R1004D/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1004D/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1004D/localizations/0/m...,NaN,0,R1004D,R1004D,NaN,protocols/r1/subjects/R1004D/experiments/catFR...
45,NaN,protocols/r1/subjects/R1013E/experiments/catFR...,protocols/r1/subjects/R1013E/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1013E/experiments/catFR...,1,NaN,0,protocols/r1/subjects/R1013E/localizations/0/m...,NaN,0,R1013E,R1013E_1,NaN,protocols/r1/subjects/R1013E/experiments/catFR...
58,NaN,protocols/r1/subjects/R1015J/experiments/catFR...,protocols/r1/subjects/R1015J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1015J/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1015J/localizations/0/m...,NaN,0,R1015J,R1015J,NaN,protocols/r1/subjects/R1015J/experiments/catFR...
64,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,0,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...
65,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,1,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,1,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,NaN,protocols/r1/subjects/R1505J/experiments/catFR...,protocols/r1/subjects/R1505J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1505J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1505J/localizations/0/m...,NaN,1,R1505J,R1505J,3.4,protocols/r1/subjects/R1505J/experiments/catFR...
2326,NaN,protocols/r1/subjects/R1515T/experiments/catFR...,protocols/r1/subjects/R1515T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1515T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1515T/localizations/0/m...,NaN,0,R1515T,R1515T,3.4,protocols/r1/subjects/R1515T/experiments/catFR...
2327,NaN,protocols/r1/subjects/R1515T/experiments/catFR...,protocols/r1/subjects/R1515T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1515T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1515T/localizations/0/m...,NaN,1,R1515T,R1515T,3.4,protocols/r1/subjects/R1515T/experiments/catFR...
2328,NaN,protocols/r1/subjects/R1515T/experiments/catFR...,protocols/r1/subjects/R1515T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1515T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1515T/localizations/0/m...,NaN,2,R1515T,R1515T,3.4,protocols/r1/subjects/R1515T/experiments/catFR...


## LOCAL run (original program)

In [151]:
import mne
from scipy.signal import firwin,filtfilt,kaiserord
import pingouin as pg
%autoreload

### PARAMS ###

selected_period = 'surrounding_recall' # surrounding_recall # whole_retrieval # encoding 
# there are three periods this code is set up to look at: periods aligned to recall, the entire retrieval period, and the encoding period
recall_type_switch = 0  # how do we select recalls?? Numbers 0:3
# 0: Original analysis taking only recalls without a recall in 2 s IRI before them
# 1: Take these same recalls, but keep only those WITH a recall within 2 s after they occur
# 2: test condition where we look at second recalls within IRI ONLY
# 3: ISOLATED only!

selected_region = HPC_labels #HPC_labels # ENT_labels+HPC_labels 

remove_soz_ictal = False # False only removes bad electrodes and not soz/ictal
min_ripple_rate = 0.1 # Hz.
max_ripple_rate = 1.0 # Hz.
max_trial_by_trial_correlation = 0.05 # if ripples correlated more than this remove them
max_electrode_by_electrode_correlation = 0.2 #???

filter_type = 'hamming' # 'butter' # Vaz used Butterworth from 80-120 Hz; Norman used Hamming from 70-180 Hz
# •Butterworth from Vaz et al: 2nd order from 80-120 ms, hilbert, select events >2 STD above mean of filtered traces.
#      Events >=25 ms long w/ max amp >3 SD were retained as ripples. Join adjacent ripples separated by <15 ms 
# •Hilbert from Norman et al: "70-180 Hz zero-lag linear-phase Hamming FIR filter w/ transition bandwidth of 5 Hz"
#      Then Hilbert, clip extreme to 4 SD, square this clipped, smooth w/ Kaiser FIR low-pass filter with 40 Hz cutoff,
#      mean and SD computed across entire experimental duration to define the threshold for event detection
#      Events from original (squared but unclipped) signal >4 SD above baseline were selected as candidate SWR events. 
#      Duration expanded until ripple power <2 SD. Events <20 ms or >200 ms excluded. Adjacent events <30 ms separation (peak-to-peak) merged.
# •Using IED detection from Vaz for 250 Hz highpass events (really 250-499). Norman uses 20-60 Hz events which is...odd.
#       See detectRipples code in module for this algorithm


# recall params
IRI = 2000 # inter-ripple interval...remove ripples within this range (keep only first one and remove those after it)
retrieval_whole_time = 30000
# encoding params
encoding_time = 2300 # actual preentation is 1.6 s + 0.75-1.0 s so keep +700 ms so can plot +500 ms
pre_encoding_time = -700 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on with a 200 ms buffer
# these aren't likely to be changed:
desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.
eeg_buffer = 300 # buffer to add to either end of IRI when processing eeg #**

### END PARAMS ###

if 'entorhinal' in selected_region and 'left hippocampus' in selected_region:
    region_name = 'HPC_ENT'
elif 'entorhinal' in selected_region:
    region_name = 'ENT'
elif 'left hippocampus' in selected_region:
    region_name = 'HPC'
if selected_period == 'surrounding_recall':
    psth_start = -IRI # only makes sense to look at period <= IRI
    psth_end = IRI # how long to grab data after recall
elif selected_period == 'whole_retrieval':
    psth_start = -IRI # doesn't have to be IRI just 2000 ms is convenient
    psth_end = IRI+retrieval_whole_time
elif selected_period == 'encoding':
    psth_start = pre_encoding_time
    psth_end = encoding_time

ripple_array = []; fr_array = []; 
trial_nums = []; 
session_ct = 0; channel_ct = 0; total_channel_ct = 0
HPC_names = []; sub_names = []; sub_sess_names = []
electrodes_per_session = []
total_lists = 0; total_recalls = 0; kept_recalls = 0
ent_elec_ct = []; sd_regions = []; not_sd_regions = []
ripple_ied_accum_ct = []
time_add_save = [];             
encoded_word_key_array = []; serialpos_array = [] # encoding info
ordinal_position_array = []

trial_by_trial_correlation = []; elec_by_elec_correlation = []
elec_ripple_rate_array = []

all_regions = []
all_atlas = []
all_pair_numbers = []
all_stein_das = []
non_locs = 0

# to get atlas coordinate info
ct=0
coord_atlases = ['avg.x','avg.corrected.x','ind.x','ind.corrected.x','mni.x']
coord_matrix = np.empty((len(exp_df),len(coord_atlases))); coord_matrix[:] = np.nan
region_matrix = np.matrix(coord_matrix,dtype='object')

for row in exp_df.itertuples(): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
        evs = reader.load('task_events')
        evs_free_recall = evs[(evs.type=='REC_WORD') & (evs.recalled==True)] # recalled word AND correct (from this list...no instrusions). 
        
        # need to remove free recalls that happened more than once        
        word_evs = evs[evs['type']=='WORD'] # get words 
        evs_free_recall = removeRepeatedRecalls(evs_free_recall,word_evs)
        
        # select which recalls??         
        [recall_selection_name,selected_recalls_idxs] = selectRecallType(recall_type_switch,evs_free_recall,IRI)
        evs_free_recall = evs_free_recall[selected_recalls_idxs]
        
        # let's make sure remaining recalls have eeg 
        evs_free_recall = evs_free_recall[evs_free_recall.eegoffset > -1]
        if len(evs_free_recall)==0: #...and that any are left!
            break
            
        # get output position in each list for this session's recalls
        session_corrected_list_ops = getOutputPositions(evs,evs_free_recall)
            
        # get pairs and loc pairs
        pairs = reader.load('pairs')
        try:
            localizations = reader.load('localization')
        except:
            localizations = []
            non_locs+=1
        
        channel = 10
        
        # get what coord_atlases have coordinates for this channel
        for i,atlas_name in enumerate(coord_atlases):
            if atlas_name in pairs:
                coord_matrix[ct,i] = pairs.iloc[channel][atlas_name] # coord_matrix will have coordinates for the 5 coord_atlases listed above
        
        # same but check for regions in atlas
        for i,atlas_name in enumerate(['dk.region','wb.region']):
            if atlas_name in pairs.iloc[channel]:
                region_matrix[ct,i] = pairs.iloc[channel][atlas_name]        
        if len(localizations) > 1:
            for i,atlas_name in enumerate(['atlases.dk','atlases.whole_brain']):
                if atlas_name in localizations:
                    region_matrix[ct,i+2] = localizations[atlas_name]['pairs'][channel]
        ct+=1
                
        # get translation from loc to pairs
        if len(localizations) > 1:
            loc_translation = Loc2PairsTranslation(pairs,localizations)
            loc_pairs = localizations.type.pairs
            num_translations = sum([loc!=' ' for loc in loc_translation])
            if len(pairs) >= len(loc_pairs):
                if num_translations != len(loc_pairs):
                    print(sub+'-'+str(session)+'-more pairs; len(pairs): '+str(len(pairs))+', len(loc_pairs): '+str(len(loc_pairs))+', trans: '+str(num_translations))
            elif len(pairs) < len(loc_pairs):
                if num_translations != len(pairs):
                    print(sub+'-'+str(session)+'-more loc_pairs; len(loc_pairs): '+str(len(loc_pairs))+', len(pairs): '+str(len(pairs))+', trans: '+str(num_translations))
        
        tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
        elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs) 
        
        # to view all loc_pairs at once: [loc_pairs[lp:lp+1] for lp in range(len(loc_pairs)-1)]
        # to view all pairs at once: [pairs[p:p+1].label for p in range(len(pairs)-1)]
        
        all_regions.extend(elec_regions)
        all_atlas.extend(atlas_type)
        all_pair_numbers.extend(pair_number)
        all_stein_das.append(has_stein_das)
        sub_sess_names.extend(np.tile(sub+'-'+str(session),len(elec_regions))) 

    except Exception as e:
        LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt') #np.save('SWRscript_FR1.npy', ['Exception', str(e.__class__), str(e)])
        continue



/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1004D/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1004D/tal/R1004D_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1013E/localizations/0/montages/1/neuroradiology/current_processed/pairs.json
/data/eeg/R1013E_1/tal/R1013E_1_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/readers/electr

R1207J-0-more pairs; len(pairs): 160, len(loc_pairs): 147, trans: 145
R1207J-1-more pairs; len(pairs): 160, len(loc_pairs): 147, trans: 145
R1207J-2-more pairs; len(pairs): 160, len(loc_pairs): 147, trans: 145
R1207J-3-more pairs; len(pairs): 160, len(loc_pairs): 147, trans: 145


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1212P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1212P/tal/R1212P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/readers/electrodes.py:236: MissingCoordinatesWarning: Could not load MNI coordinates
  "Could not load MNI coordinates"))
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1215M/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1215M/tal/R1215M_ta

R1343J-1-more loc_pairs; len(loc_pairs): 173, len(pairs): 162, trans: 148
R1347D-0-more pairs; len(pairs): 158, len(loc_pairs): 144, trans: 140
R1347D-1-more pairs; len(pairs): 158, len(loc_pairs): 144, trans: 140
R1347D-2-more pairs; len(pairs): 158, len(loc_pairs): 144, trans: 140


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1348J/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1348J/tal/R1348J_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


...

R1354E-0-more loc_pairs; len(loc_pairs): 174, len(pairs): 142, trans: 108
R1354E-1-more loc_pairs; len(loc_pairs): 174, len(pairs): 142, trans: 108
2020-09-04_15-58-09: DF Exception: Sub: R1354E, Sess: 2, TypeError, cannot convert the series to <class 'int'>, file: <ipython-input-151-6f1ba424625e>, line no: 101
R1354E-3-more loc_pairs; len(loc_pairs): 174, len(pairs): 142, trans: 108


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1361C/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1361C/tal/R1361C_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


R1361C-0-more loc_pairs; len(loc_pairs): 131, len(pairs): 126, trans: 113
R1361C-1-more loc_pairs; len(loc_pairs): 131, len(pairs): 126, trans: 113
R1361C-2-more loc_pairs; len(loc_pairs): 131, len(pairs): 126, trans: 113


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1364C/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1364C/tal/R1364C_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


R1364C-0-more pairs; len(pairs): 109, len(loc_pairs): 100, trans: 97
R1364C-1-more pairs; len(pairs): 109, len(loc_pairs): 100, trans: 97
R1366J-0-more loc_pairs; len(loc_pairs): 276, len(pairs): 251, trans: 220
R1368T-0-more loc_pairs; len(loc_pairs): 194, len(pairs): 132, trans: 81
R1370E-0-more loc_pairs; len(loc_pairs): 153, len(pairs): 126, trans: 78
R1372C-0-more loc_pairs; len(loc_pairs): 131, len(pairs): 127, trans: 115
R1372C-1-more loc_pairs; len(loc_pairs): 131, len(pairs): 127, trans: 115
R1372C-2-more loc_pairs; len(loc_pairs): 131, len(pairs): 127, trans: 115
R1374T-0-more loc_pairs; len(loc_pairs): 145, len(pairs): 124, trans: 106
R1374T-1-more loc_pairs; len(loc_pairs): 145, len(pairs): 124, trans: 106
R1375C-1-more pairs; len(pairs): 126, len(loc_pairs): 125, trans: 113
R1375C-2-more pairs; len(pairs): 126, len(loc_pairs): 125, trans: 113
R1375C-3-more pairs; len(pairs): 126, len(loc_pairs): 125, trans: 113
R1375C-4-more pairs; len(pairs): 126, len(loc_pairs): 125, tra

In [136]:
# to view which rows have data in region_matrix
# atlas order in columns: 0) pairs dk 1) pairs wb 2) loc dk 3) loc wb
column = 3 # column to check atlas order in region_mask
region_mask = [pd.isnull(row[0,0]) for row in region_matrix[:,column]] # unpack matrix
region_mask = np.invert(region_mask)
len(exp_df.subject[region_mask])
exp_df.subject[region_mask][0:20]
exp_df.subject[region_mask][-20:]
# region_matrix[:,3]

123

1122    R1198M
1125    R1200T
1754    R1341T
1755    R1341T
1757    R1342M
1761    R1345D
1762    R1345D
1763    R1345D
1764    R1345D
1765    R1345D
1770    R1346T
1771    R1346T
1772    R1346T
1773    R1347D
1774    R1347D
1775    R1347D
1782    R1349T
1783    R1350D
1784    R1350D
1785    R1350D
Name: subject, dtype: object

2048    R1423E
2050    R1425D
2051    R1425D
2052    R1425D
2058    R1427T
2059    R1427T
2060    R1427T
2063    R1431J
2064    R1433E
2066    R1436J
2070    R1438M
2071    R1441T
2072    R1441T
2073    R1441T
2077    R1443D
2078    R1443D
2094    R1446T
2095    R1447M
2097    R1448T
2100    R1449T
Name: subject, dtype: object

## Of 547 FR1 sessions from R1001P to R1467M ## 

## FR1 channel 10 ##
pairs.json:
'dk': R1001P to R1338T, 134 sessions have region name
'wb': R1030J to R1346T, 179 sessions have region name
localization.json:
'dk': R1204T to R1461T, 133 sessions have region name
'whole_brain': R1204T to R1461T, 122 sessions have region name

## FR1 channel 40 ##
pairs.json:
'dk': R1001P to R1336T, 142 sessions have region name
'wb': R1030J to R1342M, 200 sessions have region name
localization.json:
'dk': R1198M to R1449T, 133 sessions have region name
'whole_brain': R1198M to R1449T, 123 sessions have region name

## FR1 channel 70 ##
pairs.json:
'dk': R1003P to R1311T, 125 sessions have region name
'wb': R1026D to R1318N, 179 sessions have region name
localization.json:
'dk': R1173J to R1402E, 129 sessions have region name
'whole_brain': R1173J to R1402E, 122 sessions have region name


In [147]:
len(sub_sess_names)
# sub_sess_names[:20010]
np.unique(all_atlas[:20000],return_counts=True)

65257

(array(['No atlas', 'das', 'dk', 'ind', 'stein', 'wb'], dtype='<U8'),
 array([ 480,   36, 7401, 6267, 1459, 4357]))

In [152]:
sum(~np.isnan(coord_matrix[:,0])) # avg
len(coord_matrix)
# Atlas coordinates for FR1 (looking at Channel 10…channel 30 e.g. is different)
# avg: all 547 have
# avg.corrected: 132/547 subjects have (after ~R1341)
# ind: 546/547 have
# ind.corrected: 132/547 have (after ~R1341)
# mni: 410/553 have (missing ~half of first half of subjects)
# coord_matrix[:,4]

480

481

In [222]:
len(all_atlas)
len(all_regions)
# np.unique(all_atlas)
# np.unique(all_regions)
len(all_stein_das)

# what sessions have stein and das annotated?
np.array(all_stein_das)

# # how often is "MTL" atlas used?
# np.array(all_regions)[np.array(all_atlas)=='MTL_localization']

# what % of sessions have localization.json?
(len(exp_df)-non_locs)/len(exp_df)*100

65472

65472

548

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

24.95479204339964

In [223]:
# display coord_atlases with given region:
region = ' left amygdala'
len(np.array(all_atlas)[np.array(all_regions)==region])
np.array(all_atlas)[np.array(all_regions)==region]

print('left and right "hippocampus" '+str(160+93+43+118)+' electrodes') # using 'left hippocampus', ' left hippocampus' and same for right
print('left and right "amygdala" '+str(152+96+7+8)+' electrodes') # same with amygdala
(160+93+43+118)/len(all_atlas)*100
(152+96+7+8)/len(all_atlas)*100
print('For comparison: ')
print('left and right ca1 '+str(7+150+547+379)+' electrodes')
print('left and right entorhinal '+str(3+103+59+17+80+21)+' electrodes') # '"erc"', 'entorhinal' and then left/right names from below
(7+150+547+379)/len(all_atlas)*100
(3+103+59+17+80+21)/len(all_atlas)*100

152

array(['wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localization', 'wb_localization',
       'wb_localization', 'wb_localizati

left and right "hippocampus" 414 electrodes
left and right "amygdala" 263 electrodes


0.6323313782991202

0.4016984359726295

For comparison: 
left and right ca1 1083 electrodes
left and right entorhinal 283 electrodes


1.6541422287390029

0.4322458455522971

In [224]:
np.unique(np.array(sub_sess_names)[np.array(all_regions)==region])

array(['R1207J-0', 'R1207J-1', 'R1347D-2', 'R1347D-3', 'R1347D-4',
       'R1354E-0', 'R1354E-1', 'R1355T-0', 'R1355T-1', 'R1355T-2',
       'R1361C-0', 'R1361C-1', 'R1367D-0', 'R1367D-1', 'R1373T-0',
       'R1376D-0', 'R1379E-0', 'R1379E-1', 'R1383J-0', 'R1385E-1',
       'R1385E-2', 'R1385E-3', 'R1387E-0', 'R1387E-1', 'R1387E-2',
       'R1394E-0', 'R1394E-1', 'R1395M-0', 'R1395M-1', 'R1395M-2',
       'R1395M-3', 'R1395M-4', 'R1395M-5', 'R1398J-1', 'R1398J-3',
       'R1402E-0', 'R1405E-0', 'R1405E-1', 'R1414E-0', 'R1415T-0',
       'R1415T-1', 'R1423E-0', 'R1425D-0', 'R1425D-1', 'R1425D-2',
       'R1436J-1', 'R1447M-1', 'R1449T-0', 'R1449T-1', 'R1454M-0',
       'R1463E-0', 'R1463E-1', 'R1467M-0'], dtype='<U10')

In [293]:
np.unique(all_regions)
np.unique(all_atlas)

array([' left acgg anterior cingulate gyrus',
       ' left ains anterior insula', ' left amygdala',
       ' left ang angular gyrus', ' left aorg anterior orbital gyrus',
       ' left calc calcarine cortex', ' left caudate',
       ' left cerebellum exterior', ' left cerebral white matter',
       ' left co central operculum', ' left cun cuneus',
       ' left ent entorhinal area', ' left fo frontal operculum',
       ' left frp frontal pole', ' left fug fusiform gyrus',
       ' left gre gyrus rectus', ' left hippocampus',
       ' left inf lat vent', ' left iog inferior occipital gyrus',
       ' left itg inferior temporal gyrus', ' left lateral ventricle',
       ' left lig lingual gyrus', ' left lorg lateral orbital gyrus',
       ' left mcgg middle cingulate gyrus',
       ' left mfc medial frontal cortex',
       ' left mfg middle frontal gyrus',
       ' left mog middle occipital gyrus',
       ' left morg medial orbital gyrus',
       ' left mpog postcentral gyrus medial segm

array(['MTL_localization', 'No atlas', 'das', 'dk', 'dk_localization',
       'ind', 'stein', 'wb', 'wb_localization'], dtype='<U16')

In [306]:
sum('wb' in atlas for atlas in all_atlas)

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  """Entry point for launching an IPython kernel.


32293

In [307]:
np.unique(all_atlas)

array(['MTL_localization', 'No atlas', 'das', 'dk', 'dk_localization',
       'ind', 'stein', 'wb', 'wb_localization'], dtype='<U16')